In [ ]:
import random
import math
import statistics as stats
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional

import simpy


@dataclass
class MMcParams:
    lam: float = 2.0          # tasa de llegadas (λ) [clientes / unidad de tiempo]
    mu: float = 1.0           # tasa de servicio (μ)  [clientes / unidad de tiempo]
    c: int = 3                # número de servidores
    sim_time: float = 10_000  # horizonte de simulación
    seed: int = 42            # semilla RNG


@dataclass
class MMcResults:
    n_customers: int
    avg_wait_q: float
    avg_time_sys: float
    avg_q_len: float
    avg_in_system: float
    utilization: float
    raw_waits: List[float] = field(default_factory=list)
    raw_system_times: List[float] = field(default_factory=list)


class MMcMonitor:
    """
    Integra áreas (método 'área bajo la curva') para:
    - Lq(t): clientes en cola
    - L(t): clientes en sistema (cola + servicio)
    - B(t): servidores ocupados
    """
    def __init__(self, env: simpy.Environment, servers: simpy.Resource, c: int):
        self.env = env
        self.servers = servers
        self.c = c

        self.last_t = 0.0
        self.area_Lq = 0.0
        self.area_L = 0.0
        self.area_B = 0.0

    def _state(self):
        q = len(self.servers.queue)      # en cola esperando recurso
        b = self.servers.count           # en servicio
        return q, q + b, b

    def update(self):
        now = self.env.now
        dt = now - self.last_t
        if dt < 0:
            raise ValueError("El tiempo retrocedió, algo raro pasó.")
        q, L, b = self._state()

        # Integra usando el estado ANTERIOR durante [last_t, now)
        # Para eso, calculamos estado anterior con servers tal cual estaba antes del cambio.
        # Como este update se llama justo en eventos, esta aproximación funciona bien.
        # Para mejorar precisión extrema, se puede llamar update antes de cada cambio.
        # Aquí: llamamos update ANTES de modificar el estado relevante.

        # Nota: en esta implementación, la función update se llama
        # justo ANTES de los cambios críticos (request/release),
        # así que el estado actual representa el "estado anterior" del intervalo.
        self.area_Lq += q * dt
        self.area_L  += L * dt
        self.area_B  += b * dt

        self.last_t = now

    def finalize(self, sim_time: float):
        # cierra la última cola de integración hasta sim_time
        now = self.env.now
        if now < sim_time:
            self.update()         # integra hasta 'now' (sin avance real si now==last_t)
            self.env.run(until=sim_time)
        # al terminar, actualiza una última vez para integrar el último tramo
        self.update()


def exp_time(rate: float) -> float:
    """Muestra Exp(rate) con random.expovariate(rate)."""
    return random.expovariate(rate)


def customer(env: simpy.Environment, name: str, servers: simpy.Resource,
             params: MMcParams, monitor: MMcMonitor,
             waits: List[float], system_times: List[float]):
    arrival_t = env.now

    # Antes de entrar a cola (cambio de estado), integra
    monitor.update()
    with servers.request() as req:
        yield req  # espera a que haya servidor disponible
        # Ya entró a servicio: integra antes del cambio de estado
        monitor.update()

        start_service_t = env.now
        wait_q = start_service_t - arrival_t
        waits.append(wait_q)

        service_t = exp_time(params.mu)
        yield env.timeout(service_t)

        depart_t = env.now
        system_times.append(depart_t - arrival_t)

        # Antes de liberar (cambio), integra
        monitor.update()
        # al salir del with, se libera automáticamente el recurso
    # Después de liberar, integra nuevamente por consistencia
    monitor.update()


def arrivals(env: simpy.Environment, servers: simpy.Resource, params: MMcParams,
             monitor: MMcMonitor, waits: List[float], system_times: List[float]):
    i = 0
    while True:
        interarrival = exp_time(params.lam)
        yield env.timeout(interarrival)
        i += 1
        env.process(customer(env, f"C{i}", servers, params, monitor, waits, system_times))


def run_mmc(params: MMcParams) -> MMcResults:
    random.seed(params.seed)

    env = simpy.Environment()
    servers = simpy.Resource(env, capacity=params.c)  # 3 servidores => capacity=3 :contentReference[oaicite:1]{index=1}
    monitor = MMcMonitor(env, servers, params.c)

    waits: List[float] = []
    system_times: List[float] = []

    env.process(arrivals(env, servers, params, monitor, waits, system_times))
    env.run(until=params.sim_time)  # correr hasta horizonte :contentReference[oaicite:2]{index=2}

    # Finaliza integración exactamente en sim_time
    monitor.update()

    n = len(system_times)
    avg_wait_q = sum(waits) / n if n else float("nan")
    avg_time_sys = sum(system_times) / n if n else float("nan")

    avg_q_len = monitor.area_Lq / params.sim_time
    avg_in_system = monitor.area_L / params.sim_time
    utilization = (monitor.area_B / params.sim_time) / params.c  # 0..1 promedio por servidor

    return MMcResults(
        n_customers=n,
        avg_wait_q=avg_wait_q,
        avg_time_sys=avg_time_sys,
        avg_q_len=avg_q_len,
        avg_in_system=avg_in_system,
        utilization=utilization,
        raw_waits=waits,
        raw_system_times=system_times,
    )


def run_replications(base_params: MMcParams, n_rep: int = 20, seed0: int = 100) -> Dict[str, Any]:
    results = []
    for r in range(n_rep):
        p = MMcParams(**{**base_params.__dict__, "seed": seed0 + r})
        results.append(run_mmc(p))

    Wq = [res.avg_wait_q for res in results]
    W  = [res.avg_time_sys for res in results]
    U  = [res.utilization for res in results]
    Lq = [res.avg_q_len for res in results]
    L  = [res.avg_in_system for res in results]

    def mean_ci95(xs: List[float]):
        m = stats.mean(xs)
        if len(xs) < 2:
            return m, float("nan")
        s = stats.stdev(xs)
        half = 1.96 * s / math.sqrt(len(xs))  # aprox normal
        return m, half

    out = {
        "n_rep": n_rep,
        "Wq_mean_ci95": mean_ci95(Wq),
        "W_mean_ci95": mean_ci95(W),
        "Lq_mean_ci95": mean_ci95(Lq),
        "L_mean_ci95": mean_ci95(L),
        "U_mean_ci95": mean_ci95(U),
        "per_rep": results,
    }
    return out


In [ ]:
params = MMcParams(
    lam=2.4,      # λ
    mu=1.0,       # μ
    c=3,          # 3 servidores
    sim_time=50_000,
    seed=42
)

res = run_mmc(params)
res


In [ ]:
summary = run_replications(params, n_rep=30, seed0=1000)

summary["Wq_mean_ci95"], summary["W_mean_ci95"], summary["U_mean_ci95"]


((1.0770789520768955, 0.012866859876521156),
 (2.0770570721900734, 0.013187929398164946),
 (0.7993563783529956, 0.000892540841557397))